In [2]:
import tensorflow as tf
from jacovid import featurize
import numpy as np
import pandas as pd
from lib.jacques import kcqe

In [ ]:
# if we are passing loss to compile()
# option1. create a model with pinball loss 
class MyModel(tf.keras.Model):
    #https://www.tensorflow.org/api_docs/python/tf/keras/Model#example_8 
    def __init__(self, *args, **kwargs):
        super(MyModel, self).__init__(*args, **kwargs)
    
    # override default compute_loss()
    def compute_loss(self, x, y, y_pred, sample_weight):
        # x is not used
        # this pinball_loss should calculate loss based on quanile forecasts created from trainable var from fcn net
        loss += tf.add_n(self.losses)
        loss = pinball_loss(y, y_pred, sample_weight)
        self.loss_tracker.update_state(loss)
        return loss
    def reset_metrics(self):
        self.loss_tracker.reset_states()

    @property
    def metrics(self):
        return [self.loss_tracker]
    
# or only create a loss function and pass this into compile()
# https://www.tensorflow.org/guide/keras/train_and_evaluate#custom_losses 


In [ ]:
# if not passing loss to compile()
# create a model object with train_step() and train_step takes  generator as data input

# https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit#setup


fcn model #1: output a model object that could be self defined with loss function

In [15]:
def fcn_base_model(network_input, dropout_rate):
    
    expanded_input = tf.expand_dims(network_input, -1)
    input_shape = tuple(expanded_input.shape)
    x_input = tf.keras.Input(tensor=expanded_input)
   
    # 1 fcn block
    # 128
    #temporal convolution
    # , input_shape=input_shape[2:])
    x = tf.keras.layers.Conv1D(filters=10, kernel_size=10, input_shape=input_shape[2:])(x_input)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)


    # 2 fcn block
    #256
    x = tf.keras.layers.Conv1D(filters=8, kernel_size=10)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)


    # 3 fcn block
    #128
    x = tf.keras.layers.Conv1D(filters=8, kernel_size=10)(x)
    x = tf.keras.layers.Dropout(dropout_rate)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)

    # global pooling
    x = tf.keras.layers.GlobalMaxPooling2D()(x)
    # softmax
    x_output = tf.keras.layers.Activation('softmax')(x)

    model = tf.keras.Model(inputs=x_input, outputs=x_output, name="fcn_model")
    return model

In [16]:
base_network = fcn_base_model(x_train, 0.3)
#x_train_input = tf.keras.Input(tensor=x_train)

In [17]:
print(base_network.trainable_variables)

[<tf.Variable 'conv1d_3/kernel:0' shape=(10, 1, 10) dtype=float32, numpy=
array([[[ 0.1610179 , -0.15474096,  0.10120198,  0.17024288,
          0.18329427, -0.06750628,  0.00529368, -0.05769323,
          0.02594295,  0.20898688]],

       [[-0.12533943,  0.02273303,  0.22676536,  0.15856123,
         -0.04741244, -0.1605075 , -0.22990543, -0.21448542,
          0.01489241, -0.16968627]],

       [[ 0.06415614, -0.08986585,  0.0358471 , -0.19476092,
          0.21675944, -0.07891291, -0.20531149,  0.02760676,
         -0.17157035, -0.05209735]],

       [[-0.09462014,  0.04711035,  0.06105912, -0.02562852,
          0.12341088, -0.20921138, -0.09656112, -0.06923945,
          0.16715416, -0.07406117]],

       [[-0.09909797, -0.05231328, -0.10562351, -0.10069036,
         -0.03000824,  0.2152422 ,  0.12453806,  0.03384927,
         -0.09311052, -0.13795455]],

       [[-0.15304074,  0.05916265, -0.2163906 ,  0.09552574,
         -0.17879444, -0.1954892 ,  0.13774285,  0.00135715,
    

In [18]:
base_network.summary()

Model: "fcn_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(1, 9282, 28, 1)]        0         
                                                                 
 conv1d_3 (Conv1D)           (1, 9282, 19, 10)         110       
                                                                 
 dropout_3 (Dropout)         (1, 9282, 19, 10)         0         
                                                                 
 batch_normalization_3 (Batc  (1, 9282, 19, 10)        40        
 hNormalization)                                                 
                                                                 
 activation_4 (Activation)   (1, 9282, 19, 10)         0         
                                                                 
 conv1d_4 (Conv1D)           (1, 9282, 10, 8)          808       
                                                         

fcn model #2: return encodings directly (using the tutorial in overleaf)

In [5]:
def fcn_model(x_train, x_val, dropout_rate):
    # Define the tensors for the two input with new batch size
    x_train_input = tf.keras.Input(tensor=x_train,shape=(1,1,x_train.shape[2]))
    x_val_input = tf.keras.Input(tensor=x_val,shape=(1,1,x_val.shape[2]))
    
    # fcn model
    fcn_model = tf.keras.Sequential()
    # 1 fcn block
    # 128
    #temporal convolution
    # , input_shape=input_shape[2:])
    fcn_model.add(tf.keras.layers.Conv1D(filters=10, kernel_size=10))
    fcn_model.add(tf.keras.layers.Dropout(dropout_rate))
    fcn_model.add(tf.keras.layers.BatchNormalization())
    fcn_model.add(tf.keras.layers.Activation('relu'))


    # 2 fcn block
    #256
    fcn_model.add(tf.keras.layers.Conv1D(filters=8, kernel_size=20))
    fcn_model.add(tf.keras.layers.Dropout(dropout_rate))
    fcn_model.add(tf.keras.layers.BatchNormalization())
    fcn_model.add(tf.keras.layers.Activation('relu'))


    # 3 fcn block
    #128
    fcn_model.add(tf.keras.layers.Conv1D(filters=8, kernel_size=20))
    fcn_model.add(tf.keras.layers.Dropout(dropout_rate))
    fcn_model.add(tf.keras.layers.BatchNormalization())
    fcn_model.add(tf.keras.layers.Activation('relu'))

    # global pooling
    fcn_model.add(tf.keras.layers.GlobalMaxPooling1D())
    # softmax
    fcn_model.add(tf.keras.layers.Activation('softmax'))

    # shape is n_train x encoding
    # should be tensor
    x_train_encodings = fcn_model(x_train_input)
    # shape is n_val x encoding
    x_val_encodings = fcn_model(x_val_input)

    return fcn_model, x_train_encodings, x_val_encodings

In [12]:
data = pd.read_csv("/Users/yijinwang/Desktop/hosp_data_till_July.csv")
# x_train_val is 51 * 245 * 28
# y_train_val is 51 * 245
# x_T is 51 * 1 * 28

# or window size = 128
x_train_val, y_train_val, x_T = featurize.featurize_data(data = data, features = [
        {
        "fun": 'lagged_values',
        "args": {'target_var': "inc_hosp", 'window_size': 28}
        },
    ])

In [13]:
kcqe_obj = kcqe.KCQE(p=28)
single_batch_generator = kcqe_obj.single_batch_generator(x_train_val=x_train_val, y_train_val=y_train_val, block_size = 21)

In [14]:
# x_val is 1 * 1071 * 28
# x_train is 1 * 9282 * 28
# y_val is 1 * 1071
# y_train is 1 * 9282
x_val, x_train, y_val, y_train = next(single_batch_generator)

In [9]:
model, x_train_encodings, x_val_encodings = fcn_model(x_train, x_val, 0.3)

In [11]:
x_val_encodings

<KerasTensor: shape=(1, 8) dtype=float32 (created by layer 'sequential')>

In [22]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (1, None, 10)             2810      
                                                                 
 dropout_3 (Dropout)         (1, None, 10)             0         
                                                                 
 batch_normalization_3 (Batc  (1, None, 10)            40        
 hNormalization)                                                 
                                                                 
 activation_4 (Activation)   (1, None, 10)             0         
                                                                 
 conv1d_4 (Conv1D)           (1, None, 8)              1608      
                                                                 
 dropout_4 (Dropout)         (1, None, 8)              0         
                                                      

In [23]:
x_train.shape

TensorShape([1, 9282, 28])

In [9]:
input_shape = (4, 28, 3)
input_shape[:-1]

(4, 28)

In [22]:
(x_train.shape[0],x_train.shape[1])

(1, 9282)

In [13]:
x_train_input = tf.keras.Input(tensor=x_train, shape=(1,1,x_train.shape[2]))

In [14]:
x_train_input.shape

TensorShape([1, 9282, 28])

In [42]:
fcn_model = tf.keras.Sequential()

In [43]:
fcn_model.add(tf.keras.layers.Conv1D(filters=10, kernel_size=10))
fcn_model.add(tf.keras.layers.Dropout(0.3))
fcn_model.add(tf.keras.layers.BatchNormalization())
fcn_model.add(tf.keras.layers.Activation('relu'))

#1, 9273, 10

fcn_model.add(tf.keras.layers.Conv1D(filters=8, kernel_size=20))
fcn_model.add(tf.keras.layers.Dropout(0.3))
fcn_model.add(tf.keras.layers.BatchNormalization())
fcn_model.add(tf.keras.layers.Activation('relu'))

In [44]:
fcn_model(x_train_input)

<KerasTensor: shape=(1, 9254, 8) dtype=float32 (created by layer 'sequential_4')>